In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df.drop(labels='id',axis=1,inplace=True)

In [5]:
# independent and dependent features
X=df.drop('price',axis=1)
y=df.price

In [6]:
numerical_columns=X.select_dtypes(exclude='object').columns
categorical_columns=X.select_dtypes(include='object').columns


In [7]:
numerical_columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [8]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [9]:
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [10]:
numerical_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

categorical_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

In [11]:
preprocessor=ColumnTransformer([
    ('num_pipeline',numerical_pipeline,numerical_columns),
    ('cat_pipeline',categorical_pipeline,categorical_columns)
])

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=10)

In [13]:
X_train

,carat,cut,color,clarity,depth,table,x,y,z
30702,0.43,Very Good,D,VS1,62.6,56.0,4.78,4.83,3.01
71255,1.62,Premium,G,VS2,60.4,58.0,7.58,7.65,4.59
7383,0.44,Premium,D,VS1,60.1,58.0,4.95,4.88,2.96
130477,0.82,Ideal,G,SI2,61.3,56.0,6.05,6.03,3.71
23690,0.53,Ideal,G,VVS1,62.5,56.0,5.18,5.16,3.23
...,...,...,...,...,...,...,...,...,...
105595,1.55,Ideal,F,SI1,62.0,55.0,7.37,7.45,4.58
93553,0.90,Premium,G,SI2,60.2,59.0,6.34,6.23,3.77
181568,1.01,Very Good,D,VVS2,60.3,56.0,6.50,6.55,3.93
94735,0.41,Ideal,H,SI1,61.7,56.0,4.80,4.77,2.95


In [14]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())


In [15]:
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.780167,0.719352,-0.637086,-0.843944,-0.808252,-0.760667,-1.139986,-1.550963,0.682694
1,1.793874,-1.307964,0.403474,1.681406,1.751577,1.530872,-0.134981,0.297649,0.017348
2,-0.758536,-1.584416,0.403474,-0.690619,-0.762865,-0.833184,-0.134981,-1.550963,0.682694
3,0.063426,-0.478607,-0.637086,0.301483,0.281037,0.254572,0.870024,0.297649,-1.313344
4,-0.563861,0.627201,-0.637086,-0.483179,-0.508697,-0.441592,0.870024,0.297649,2.013386
...,...,...,...,...,...,...,...,...,...
135496,1.642460,0.166447,-1.157367,1.492004,1.570029,1.516369,0.870024,-0.318555,-0.647998
135497,0.236471,-1.492265,0.923755,0.563037,0.462585,0.341592,-0.134981,0.297649,-1.313344
135498,0.474408,-1.400114,-0.637086,0.707342,0.753062,0.573647,-1.139986,-1.550963,1.348040
135499,-0.823428,-0.110005,-0.637086,-0.825905,-0.862716,-0.847687,0.870024,0.913854,-0.647998


In [16]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
import numpy as np
def evaluate_performance(y_actual,y_pred):
    mae = mean_absolute_error(y_actual,y_pred)
    mse = mean_squared_error(y_actual,y_pred)
    rmse = np.sqrt(mse)
    Score = r2_score(y_actual,y_pred)
    return mae,mse, rmse, Score

In [18]:
models=[('linear_regressor',LinearRegression()),
        ('lasso',Lasso()),
        ('ridge',Ridge()),
        ('elastic_net',ElasticNet()),
        ('knn',KNeighborsRegressor()),
        ('tree',DecisionTreeRegressor())
        ]

for i in range(len(models)):
    model=models[i][1]
    model.fit(X_train,y_train)

    y_pred=model.predict(X_test)

    mae,mse,rmse,r2score=evaluate_performance(y_test,y_pred)

    print(models[i][0])
    print('Model Performance')
    print('mae',mae)
    print('mse',mse)
    print('rmse',rmse)
    print('r2_score',r2score)
    print('-'*40)




linear_regressor
Model Performance
mae 675.2796042191958
mse 1029839.1822413101
rmse 1014.8099241933487
r2_score 0.937049390893374
----------------------------------------
lasso
Model Performance
mae 676.3733606162151
mse 1030113.3268792456
rmse 1014.94498711962
r2_score 0.9370326333527415
----------------------------------------
ridge
Model Performance
mae 675.3022121068825
mse 1029810.1049045292
rmse 1014.7955975981218
r2_score 0.937051168293276
----------------------------------------
elastic_net
Model Performance
mae 1064.3039298403096
mse 2367021.380551036
rmse 1538.5127170586002
r2_score 0.8553119358405031
----------------------------------------
knn
Model Performance
mae 350.74615993938556
mse 456062.8812846122
rmse 675.3242786133283
r2_score 0.9721224083693273
----------------------------------------
tree
Model Performance
mae 424.5449183771869
mse 692386.8708045376
rmse 832.0978733325411
r2_score 0.9576767169028112
----------------------------------------


In [19]:
## Knn is giving better results